In [1]:
import numpy as np

WHITE_KING = 2
WHITE = 1
EMPTY = 0
BLACK = -1
BLACK_KING = -2

NUM_OF_NON_CAPTURE_MOVES = 50

In [2]:
class Board:
  def __init__(self, player=WHITE, size=4, board_config=None, non_capture_moves=0):
    if size < 3:
      size = 4
    if size % 2 != 0:
      size += 1

    if board_config == None:
      self.board = self.setup_board(size)
    else:
      self.board = board_config

    self.player = player
    self.size = size
    self.white_pieces = self.get_pieces_position(WHITE)
    self.black_pieces = self.get_pieces_position(BLACK)

    self.non_capture_moves = non_capture_moves

  def setup_board(self, size):
    board = []

    for row in range(size):
      if row < size / 2:
        piece_color = BLACK
      else:
        piece_color = WHITE

      for col in range(size):
        if row == size / 2 or row == size / 2 - 1:
          board.append(EMPTY)
          continue

        if row % 2 == 0:
          if col % 2 == 0:
            board.append(EMPTY)
          else:
            board.append(piece_color)
        else:
          if col % 2 == 0:
            board.append(piece_color)
          else:
            board.append(EMPTY)

    return board

  def get_pieces_position(self, player):
    pieces = []
    for pos, piece in enumerate(self.board):
      if piece == player or piece == player * 2:
        pieces.append(pos)

    return pieces
  
  def create_new_board_from_move(self, move):
    new_board = self.board.copy()

    piece_to_move = move[0]
    new_pos = move[1]
    dist = abs(piece_to_move - new_pos)

    capture = False

    piece = self.board[piece_to_move]

    if (new_pos < self.size or new_pos >= (self.size * (self.size - 1))) and abs(piece) == 1:
      piece *= 2

    new_board[new_pos] = piece
    new_board[piece_to_move] = EMPTY    
    
    if dist > self.size + 1:       
      captured_piece = int((piece_to_move + new_pos) / 2)
      new_board[captured_piece] = EMPTY

      capture = True

    if capture:
      non_capture_moves = 0
    else:
      non_capture_moves = self.non_capture_moves + 1

    return Board(size=self.size, player=self.player * -1, board_config=new_board, non_capture_moves=non_capture_moves)

  def get_possible_moves(self):
    if self.player == WHITE:
      pieces = self.white_pieces
    else:
      pieces = self.black_pieces

    moves = []
    moves.extend(self.get_regular_moves(pieces))
    moves.extend(self.get_capture_moves(pieces))

    return moves

  def get_regular_moves(self, pieces):
    moves = []

    for piece_pos in pieces:
      neighbours = self.get_neighbouring_squares(piece_pos)
      for n in neighbours:
        if not self.check_moving_forward(piece_pos, n) and abs(self.board[piece_pos]) != 2: # testa se a peça tá voltando (só p/ peças normais (1 e -1))
          continue

        if self.board[n] == EMPTY: # qualquer outro caso (peça avançando, ou rei se movendo)
          moves.append((piece_pos, n))

    return moves

  def get_capture_moves(self, pieces):
    moves = []

    for piece_pos in pieces:
      if self.board[piece_pos] > 0:
        player = WHITE
      else:
        player = BLACK

      neighbours = self.get_neighbouring_squares(piece_pos)
      for n in neighbours:
        if not self.check_moving_forward(piece_pos, n) and abs(self.board[piece_pos]) != 2:
          continue

        if (player == WHITE and self.board[n] < 0) or (player == BLACK and self.board[n] > 0):
          sq_after = self.get_position_after_capture(piece_pos, n)
          if sq_after >= 0 and self.board[sq_after] == EMPTY:
              moves.append((piece_pos, sq_after))
          
    return moves


  def get_neighbouring_squares(self, position):
    neighbours = []

    up = position - self.size
    down = position + self.size
    left = -1
    right = 1

    if up >= 0:
      if (up + left) % self.size != self.size - 1:
        neighbours.append(up + left)
      
      if (up + right) % self.size != 0:
        neighbours.append(up + right)

    if down < self.size ** 2:
      if (down + left) % self.size != self.size - 1:
        neighbours.append(down + left)

      if (down + right) % self.size !=  0:
        neighbours.append(down + right)

    return neighbours

  def check_moving_forward(self, piece_pos, target_pos):
    if self.board[piece_pos] == WHITE and target_pos > piece_pos:
      return False
    elif self.board[piece_pos] == BLACK and target_pos < piece_pos:
      return False

    return True

  def get_position_after_capture(self, piece_pos, target_pos):
    if target_pos < self.size or target_pos >= self.size * (self.size - 1): # primeira e ultima fileiras
      return -1
    
    if target_pos % self.size == 0 or (target_pos + 1) % self.size == 0: # primeira e ultima colunas
      return -1

    return piece_pos - (piece_pos - target_pos) * 2


  def print_board(self):
    for row in range(self.size):
      for col in range(self.size):
        pos = row * self.size + col
        sq = self.board[pos]

        if sq == BLACK:
          print("BP", end='\t')
        elif sq == BLACK_KING:
          print("BK", end='\t')
        elif sq == EMPTY:
          print("-", end='\t')
        elif sq == WHITE:
          print("WP", end='\t')
        elif sq == WHITE_KING:
          print("WK", end='\t')

      print('')
    

  def end(self):
    if len(self.get_possible_moves()) == 0 or self.non_capture_moves > NUM_OF_NON_CAPTURE_MOVES:
      return True
    
    return False

  def get_result(self):
    if len(self.get_possible_moves()) == 0:
      if self.player == WHITE: # se brancas sem movimento, derrota
        return -1
      else:                   # se pretas sem movimento, vitória
        return 1

    if self.non_capture_moves > NUM_OF_NON_CAPTURE_MOVES \
      or (len(self.black_pieces) == 1 and len(self.white_pieces) == 1): # se muito tempo sem capturas, empate
      return 0

    return None

In [3]:
class SearchTree:
  def __init__(self, board):
    self.board = board
    self.children = []
    self.best_move = None

  def get_allowed_moves(self):
    return self.board.get_possible_moves()

  def get_children(self):
    return self.children

  def get_last_child(self):
    return self.children[-1]

  def create_child(self, move):
    board = self.board.create_new_board_from_move(move)
    self.children.append(SearchTree(board))

  def get_position_score(self):
    count_max = len(self.board.white_pieces)
    count_min = len(self.board.black_pieces)

    return count_max - count_min

  def get_result(self):
    return self.board.get_result()

In [4]:
def minimax(root, alfa=float('-inf'), beta=float('inf'), search_depth=3, player='max'):
  r = root.get_result()

  if r == 1:
    return float('inf') # vitória
  elif r == -1:
    return float('-inf') # derrota
  elif r == 0:
    return 0

  if search_depth == 0:
    return root.get_position_score()

  moves = root.get_allowed_moves()

  if player is 'max':
    value = float('-inf')

    for move in moves:
      root.create_child(move)
      next_node = root.get_last_child()

      move_val = minimax(next_node, alfa, beta, search_depth=search_depth - 1, player='min')

      if move_val >= value:
        value = move_val
        best_move = move

      if value >= beta:
        break
      
      alfa = max(alfa, value)
  else:
    value = float('inf')

    for move in moves:
      root.create_child(move)
      next_node = root.get_last_child()

      move_val = minimax(next_node, alfa, beta, search_depth=search_depth - 1, player='max')

      if move_val <= value:
        value = move_val
        best_move = move

      if value <= alfa:
        break
      
      beta = min(beta, value)

  root.best_move = best_move
  return value

In [5]:
def play(size=4, search_depth=3, print_board=False):
  played_moves = []
  player = 'max'
  game = SearchTree(Board(size=size))

  while True:
    value = minimax(game, search_depth=search_depth, player=player)
    print("Move: ")
    print(value, game.best_move)

    move = game.best_move

    played_moves.append(move)
    game = SearchTree(game.board.create_new_board_from_move(move))
    
    if print_board:
      game.board.print_board()

    if game.get_result() is not None:
      print("Fim")
      print(played_moves)
      break    

    if player == 'max':
      player = 'min'
    else:
      player = 'max'

  return game

In [ ]:
# O tabuleiro é uma lista, as posições são contadas a partir do canto superior esquerdo (a posição 0)
# acabam no canto inferior direito (a posição tamanho² - 1)

# as peças pretas ficam em cima, e as brancas começam
# As duas fileiras do meio são vazias

In [10]:
game = play(size=4, search_depth=3, print_board=True)

Move: 
0 (14, 11)
-	BP	-	BP	
-	-	-	-	
-	-	-	WP	
WP	-	-	-	
Move: 
-1 (3, 6)
-	BP	-	-	
-	-	BP	-	
-	-	-	WP	
WP	-	-	-	
Move: 
-1 (12, 9)
-	BP	-	-	
-	-	BP	-	
-	WP	-	WP	
-	-	-	-	
Move: 
-inf (6, 12)
-	BP	-	-	
-	-	-	-	
-	-	-	WP	
BK	-	-	-	
Move: 
-inf (11, 6)
-	BP	-	-	
-	-	WP	-	
-	-	-	-	
BK	-	-	-	
Move: 
-inf (12, 9)
-	BP	-	-	
-	-	WP	-	
-	BK	-	-	
-	-	-	-	
Move: 
-inf (6, 3)
-	BP	-	WK	
-	-	-	-	
-	BK	-	-	
-	-	-	-	
Move: 
-inf (1, 4)
-	-	-	WK	
BP	-	-	-	
-	BK	-	-	
-	-	-	-	
Move: 
-inf (3, 6)
-	-	-	-	
BP	-	WK	-	
-	BK	-	-	
-	-	-	-	
Move: 
-inf (9, 3)
-	-	-	BK	
BP	-	-	-	
-	-	-	-	
-	-	-	-	
Fim
[(14, 11), (3, 6), (12, 9), (6, 12), (11, 6), (12, 9), (6, 3), (1, 4), (3, 6), (9, 3)]
